In [1]:
import os
import glob
import json
import random
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Định nghĩa đường dẫn (Dựa trên cấu trúc thư mục của bạn)
DATA_ROOT = "../data"
RAW_IMG_DIR = os.path.join(DATA_ROOT, "raw/images")
RAW_LABEL_DIR = os.path.join(DATA_ROOT, "raw/annotations") # Hoặc boxs_and_transcript
PROCESSED_DIR = os.path.join(DATA_ROOT, "processed")

# Tạo thư mục processed nếu chưa có
os.makedirs(PROCESSED_DIR, exist_ok=True)

print("Đường dẫn ảnh:", RAW_IMG_DIR)
print("Đường dẫn nhãn:", RAW_LABEL_DIR)

Đường dẫn ảnh: ../data\raw/images
Đường dẫn nhãn: ../data\raw/annotations


In [2]:
# Định nghĩa các nhãn chúng ta quan tâm
# LayoutLM cần label dạng số.
label2id = {
    "OTHER": 0,
    "SELLER": 1,
    "ADDRESS": 2,      # Dữ liệu có thì cứ giữ, sau này filter sau cũng được
    "TIMESTAMP": 3,
    "TOTAL_COST": 4
}

id2label = {v: k for k, v in label2id.items()}

print("Danh sách nhãn:", label2id)

Danh sách nhãn: {'OTHER': 0, 'SELLER': 1, 'ADDRESS': 2, 'TIMESTAMP': 3, 'TOTAL_COST': 4}


In [3]:
def normalize_box(box, width, height):
    """
    Chuẩn hóa tọa độ box về thang đo 0-1000
    box: [x_min, y_min, x_max, y_max]
    """
    return [
        int(1000 * (box[0] / width)),
        int(1000 * (box[1] / height)),
        int(1000 * (box[2] / width)),
        int(1000 * (box[3] / height))
    ]

def convert_quad_to_box(quad):
    """
    Chuyển đổi 8 tọa độ (4 điểm) thành 4 tọa độ (xmin, ymin, xmax, ymax)
    Input: [x1, y1, x2, y2, x3, y3, x4, y4]
    """
    # Lấy các giá trị x và y riêng lẻ
    x_coords = [quad[0], quad[2], quad[4], quad[6]]
    y_coords = [quad[1], quad[3], quad[5], quad[7]]
    
    return [
        min(x_coords), # x_min
        min(y_coords), # y_min
        max(x_coords), # x_max
        max(y_coords)  # y_max
    ]

In [4]:
def process_annotation(file_path, img_dir):
    """
    Đọc file annotation và ảnh tương ứng, trả về dictionary dữ liệu
    """
    filename = os.path.basename(file_path)
    # Giả sử tên file ảnh giống tên file label nhưng khác đuôi (vd: abc.tsv -> abc.jpg)
    # Bạn cần kiểm tra đuôi ảnh thực tế là .jpg hay .png
    base_name = os.path.splitext(filename)[0]
    
    # Tìm file ảnh (thử cả jpg và png)
    img_path = os.path.join(img_dir, base_name + ".jpg")
    if not os.path.exists(img_path):
        img_path = os.path.join(img_dir, base_name + ".png")
        
    if not os.path.exists(img_path):
        return None # Không tìm thấy ảnh thì bỏ qua

    # Đọc ảnh để lấy kích thước
    try:
        image = Image.open(img_path)
        width, height = image.size
    except Exception as e:
        print(f"Lỗi đọc ảnh {img_path}: {e}")
        return None

    words = []
    boxes = []
    labels = []

    try:
        # Đọc file CSV/TSV
        # Dữ liệu bạn gửi có vẻ không chuẩn CSV hoàn toàn (text có thể chứa dấu phẩy)
        # Nên ta đọc từng dòng và xử lý thủ công cho an toàn
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip()
            if not line: continue
            
            parts = line.split(',')
            
            # Cấu trúc: index, 8 coords..., text..., label
            # Ít nhất phải có index(1) + 8 coords + label(1) = 10 phần tử
            if len(parts) < 10: continue
            
            # Lấy 8 tọa độ (từ index 1 đến 9)
            try:
                quad = [int(float(p)) for p in parts[1:9]]
            except ValueError:
                continue # Bỏ qua dòng lỗi tọa độ

            # Label là phần tử cuối cùng
            label_name = parts[-1].strip()
            
            # Text là phần ở giữa (xử lý trường hợp text chứa dấu phẩy)
            text_content = ",".join(parts[9:-1]).strip()
            
            # Nếu text rỗng, có thể bỏ qua hoặc để placeholder
            if not text_content:
                continue 

            # Map label name sang ID
            label_id = label2id.get(label_name, 0) # Mặc định là OTHER (0) nếu không tìm thấy

            # Chuyển đổi tọa độ và chuẩn hóa
            box = convert_quad_to_box(quad)
            norm_box = normalize_box(box, width, height)

            words.append(text_content)
            boxes.append(norm_box)
            labels.append(label_id)

        return {
            "id": base_name,
            "tokens": words,
            "bboxes": boxes,
            "ner_tags": labels,
            "image_path": img_path
        }

    except Exception as e:
        print(f"Lỗi xử lý file {file_path}: {e}")
        return None

In [5]:
# Lấy danh sách tất cả file annotation (giả sử đuôi .tsv hoặc .txt hoặc .csv)
# Bạn hãy đổi đuôi file phù hợp với dữ liệu thật (trong ví dụ bạn gửi là csv format)
all_anno_files = glob.glob(os.path.join(RAW_LABEL_DIR, "*.tsv")) 
# Nếu file của bạn không có đuôi .tsv mà là .txt hay .csv thì đổi ở trên nhé.

print(f"Tìm thấy {len(all_anno_files)} file annotations.")

dataset = []
for file_path in tqdm(all_anno_files, desc="Processing data"):
    data_item = process_annotation(file_path, RAW_IMG_DIR)
    if data_item:
        # Kiểm tra dữ liệu hợp lệ (phải có token)
        if len(data_item['tokens']) > 0:
            dataset.append(data_item)

print(f"Đã xử lý thành công: {len(dataset)} mẫu dữ liệu.")

Tìm thấy 1151 file annotations.


Processing data: 100%|██████████| 1151/1151 [00:21<00:00, 52.78it/s]

Đã xử lý thành công: 1151 mẫu dữ liệu.


In [6]:
# Chia tập train/test
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

print(f"Số lượng tập Train: {len(train_data)}")
print(f"Số lượng tập Validation: {len(val_data)}")

# Lưu ra file JSON
def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

save_json(train_data, os.path.join(PROCESSED_DIR, "train.json"))
save_json(val_data, os.path.join(PROCESSED_DIR, "validation.json"))

# Lưu luôn label map để dùng sau này
save_json(label2id, os.path.join(PROCESSED_DIR, "label_map.json"))

print("Hoàn tất! Dữ liệu đã được lưu tại:", PROCESSED_DIR)

Số lượng tập Train: 920
Số lượng tập Validation: 231
Hoàn tất! Dữ liệu đã được lưu tại: ../data\processed
